In [ ]:
# 클러스터링 번호 1번이 식품 관련이기 때문에
# 1만 클러스터링

# 2차 클러스터링 코드
# HuggingFace 사용을 위해 터미널에서 추가하기 위한 것
# pip install -U sentence-transformers

# Sentence-Transformers
from sentence_transformers import SentenceTransformer
sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]

model = SentenceTransformer('jhgan/ko-sroberta-multitask')
embeddings = model.encode(sentences)

# HuggingFace Transformers
# 클러스터링을 하기 전 임베딩을 위한...
from transformers import AutoTokenizer, AutoModel
import torch

# 데이터 클러스터링을 위한 사이킷런 설치
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# pip install scikit-learn

from sklearn.cluster import DBSCAN
import json # json 파일을 읽고 쓰기 위한
import numpy as np
import pandas as pd
import pickle   # 모델을 저장하기 위한

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# 1차 클러스터링 된 데이터 가져오기
year = 2022
newscount_end = 25
df = pd.read_csv('Clustered/' + str(year) + '-news-clustered-fullver1.csv')


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sroberta-multitask')

second_vector = [] # 차원 정보를 저장

# 필터링 된 데이터 가져오기
# filtered_data = "data_filtered/ssafy_dataset_news_" + str(year) + ".csv-" + str(i) + ".json"
#filtered_data = "2" + str(year) + ".csv-" + str(i) + ".json"
count = 0
# 필터링 된 데이터를 파일로 가져오기
# Sentences we want sentence embeddings for
#sentences = ['This is an example sentence', 'Each sentence is converted']  
# with open(filtered_data) as file:
#     filtered = json.load(file)  # json 파일 읽어오기(배열로 1개씩 가져올 수 있음)

clustered_news = df.loc[df['cluster_number'] == 1, :]
print(len(clustered_news))
clust_news = clustered_news['filtered_news']
clust_num = clustered_news['cluster_number']


# 1차 클러스터링 된 정보를 바탕으로 2차 클러스터링
for num, article in zip(clust_num, clust_news):
    # Tokenize sentences
    encoded_input = tokenizer(article, padding = True, truncation = True, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    # 차원축소를 위한 코드(사이킷런은 2차원까지만 알아먹기 때문에)
    dimension_down = np.squeeze(sentence_embeddings, axis = 0)

    # 여기까지가 임베딩을 위한 코드

    count+= 1
    second_vector.append(dimension_down)

    if count>=300:
        print("현재 데이터 "+str(count)+"개 실행 완료")
        count = 0


print("")
print("!! 모든 데이터 필터링 완료 !!")


# 클러스터링을 위한 모델, DBSCAN을 사용, cosine 계산 수식 사용
second_filtered_model = DBSCAN(eps = 0.15, min_samples = 4, metric = "cosine")
# eps 0.15인게 그나마 나음
# 아니면 0.185, 샘플3
# 0.18, 샘플 4
# 0.127, 3
# 학습된것을 fit을 사용해 모델로 따로 저장(저장 안하려면 fit_predict 사용하면 됨)
second_result = second_filtered_model.fit_predict(second_vector)
# save_second = second_filtered_model.fit(second_vector)


# with open('saved_model_2nd_fullver1', 'wb') as f:
#     pickle.dump(save_second, f)
# print("학습 모델 저장")

# # # 모델을 불러 올 때
# # with open('saved_model', 'rb') as f:
# #     mod = pickle.load(f)
# # # 이후 mod 로 모델 사용하면 됨

final_filter = []
cluster_num = []
for vec, clust, fil in zip(second_vector, second_result, clust_news):
    if clust == -1:
        continue
    else:
        # print("cluster num: " + str(clust))
        # print(fil)
        final_filter.append(fil)
        cluster_num.append(clust)

df = pd.DataFrame({'cluster_number': cluster_num, 'filtered_news': final_filter})
df.sort_values(by=['cluster_number'], inplace=True)
csv_path = "Clustered/2022-2nd-full-clustered-ver2.csv"
df.to_csv(csv_path, index=False)
print("데이터 클러스터링 완료")